In [ ]:
# 5-fold cross validation with independent datasets 
import numpy
from numpy import loadtxt
from tensorflow import keras
from keras.layers import Dense
from keras.layers import Dropout
from tensorflow.keras import layers
from keras.models import Sequential
from keras.optimizers import Adam as opt
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score

In [ ]:
# fix random seed for reproducibility
dataset = numpy.loadtxt(r"E:\MS DATA\feature\Hybrid_Features.csv", delimiter=",")

In [ ]:
X = dataset[:,0:319]
Y = dataset[:,320]
cvscores=[]

In [ ]:
# create folds
folds = list(KFold(n_splits=5, shuffle=True).split(X, Y ))

for j, (train, test) in enumerate(folds):
    model = Sequential()
    model.add(Dropout(0.4))
    model.add(Dense(96, kernel_initializer='he_uniform', activation='relu', input_dim=319))
    #model.add(Dropout(0.4))
    model.add(Dense(64, kernel_initializer='he_uniform', activation='relu'))
   # model.add3(Dropout(0.01))
    model.add(Dense(32, kernel_initializer='he_uniform', activation='relu'))
   # model.add(Dropout(0.01))
    model.add(Dense(16, kernel_initializer='he_uniform', activation='relu'))
   # model.add(Dropout(0.01))
    model.add(Dense(8, kernel_initializer='he_uniform', activation='relu'))
    model.add(Dense(units=1, kernel_initializer='he_uniform', activation='sigmoid'))

    opt = keras.optimizers.Adam(learning_rate=0.01)
    model.compile(loss='binary_crossentropy', optimizer='Adam',metrics=['accuracy'])
    
    
    print('\nFold = ',j)
    X_train_cv = X[train]
    y_train_cv = Y[train]
    X_test_cv = X[test]
    y_test_cv= Y[test]
    

	# Fit the model
    history = model.fit(X_train_cv, y_train_cv, epochs=32, batch_size=8, verbose=0)
# evaluate the model 
    scores = model.evaluate(X_test_cv, y_test_cv, verbose=0)
    print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
    cvscores.append(scores[1] * 100)
print("Test Accuracy","%.2f%%(+/- %.2f%%)" % (numpy.mean(cvscores), numpy.std(cvscores)))
_, train_acc = model.evaluate(X_train_cv, y_train_cv)
print('Train Accuracy: %.2f' % (train_acc*100))

In [ ]:
_, train_acc = model.evaluate(X_train_cv, y_train_cv)
print('Train Accuracy: %.2f' % (train_acc*100))

In [ ]:
#Predicting the test set results   # 86.04%(+/- 14.76%)
y_pred = model.predict(X_test_cv)
y_pred = (y_pred>0.5)

In [ ]:
# calculate accuracy
from sklearn import metrics
print(metrics.accuracy_score(y_test_cv, y_pred))

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test_cv, y_pred)

In [ ]:
#extracting true_positives, false_positives, true_negatives, false_negatives
TN, FP, FN, TP = confusion_matrix(y_test_cv, y_pred).ravel()
print("True Negatives: ",TN)
print("False Positives: ",FP)
print("False Negatives: ",FN)
print("True Positives: ",TP)
confusion = metrics.confusion_matrix(y_test_cv, y_pred)
print(confusion)
#[row, column]
TP = confusion[1, 1]
TN = confusion[0, 0]
FP = confusion[0, 1]
FN = confusion[1, 0]

In [ ]:
#Precision 
Precision = TP/(TP+FP) 
print("Precision {:0.2f}".format(Precision))
#Recall 
Recall = TP/(TP+FN) 
print("Recall {:0.2f}".format(Recall))

In [ ]:
#TEST ACCURACY
_, test_acc = model.evaluate(X_test_cv, y_test_cv, verbose=0)
print("Test Accuracy","%.2f%%(+/- %.2f%%)" % (numpy.mean(cvscores), numpy.std(cvscores)))

#Sensitivity
Sensitivity = TP/(FN+TP)
print("Sensitivity: {:2f}".format(Sensitivity))

#Specificity 
Specificity = TN/(TN+FP)
print("Specificity {:2f}".format(Specificity))

#MCC
from sklearn.metrics import matthews_corrcoef, confusion_matrix
confusion_matrix(y_test_cv, y_pred)
MCC = matthews_corrcoef(y_test_cv, y_pred)
print("MCC: {:2f}".format(MCC))

#F1 Score
f1 = (2*Precision*Recall)/(Precision + Recall)
print("F1 Score: {:2f}".format(f1))

ROC = metrics.roc_auc_score(y_test_cv, y_pred)
print("ROC_ACC: {:2f}".format(ROC))

#Precision 
Precision = TP/(TP+FP) 
print("Precision: {:2f}".format(Precision))

#Recall 
#Recall 
Recall = TP/(TP+FN) 
print("Recall {:2f}".format(Recall))

In [ ]:
import scikitplot as skplt
import matplotlib.pyplot as plt
skplt.metrics.plot_confusion_matrix(y_test_cv, y_pred, normalize=True)
plt.show()

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import pyplot
# summarize history for loss
# plot loss during training
#pyplot.subplot(211)
plt.figure(figsize=(12,8))
pyplot.plot(history.history['loss'])
plt.xlabel('Epoch')
plt.ylabel('Loss')
pyplot.legend()

In [ ]:
from matplotlib import pyplot
# plot loss during training
#plt.title('Loss / Mean Squared Error')
plt.figure(figsize=(10,10))
pyplot.plot(history.history['loss'])
plt.xlabel('Epoch')
plt.ylabel('Loss')
pyplot.legend()

In [ ]:
import matplotlib.pyplot as plt 
plt.figure(figsize=(10,10))
plt.plot(history.history['accuracy'])     
plt.xlabel('epoch')  
plt.legend(loc='upper left')  

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(10,10))
plt.plot(history.history['accuracy'])      
plt.xlabel('epoch')
plt.ylabel('accuracy') 
plt.legend(loc='upper left')  
plt.show()

In [ ]:
from sklearn.metrics import roc_curve, auc
false_positive_rate, true_positive_rate, thresholds = roc_curve(Y[test], y_pred)
roc_auc = auc(false_positive_rate, true_positive_rate)
roc_auc
import matplotlib.pyplot as plt
plt.figure(figsize=(10,10))
#plt.title('Receiver Operating Characteristic')
plt.plot(false_positive_rate,true_positive_rate, color='green',label = 'PsesNC' % roc_auc)
plt.legend(loc = 'lower right')
#plt.plot([0, 1], [0, 1],linestyle='--')
plt.axis('tight')
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn import svm
from sklearn import metrics

plt.figure(figsize=(10,10))


X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=.33,
                                                    random_state=0)
rf = RandomForestClassifier(max_features=5, n_estimators=100)
rf.fit(X_train, Y_train)

nb = GaussianNB()
nb.fit(X_train, Y_train)

knn = KNeighborsClassifier()
knn.fit(X_train, Y_train)



#Create a svm Classifier
clf = svm.SVC(kernel='linear') # Linear Kernel

#Train the model using the training sets
clf.fit(X_train, Y_train)

r_probs = [0 for _ in range(len(Y_test))]
rf_probs = rf.predict_proba(X_test)
nb_probs = nb.predict_proba(X_test)
knn_probs = knn.predict_proba(X_test)
y_pred = clf.predict(X_test)


rf_probs = rf_probs[:, 1]
nb_probs = nb_probs[:, 1]
knn_probs = knn_probs[:, 1]


from sklearn.metrics import roc_curve, roc_auc_score

r_auc = roc_auc_score(Y_test, r_probs)
rf_auc = roc_auc_score(Y_test, rf_probs)
nb_auc = roc_auc_score(Y_test, nb_probs)
knn_auc = roc_auc_score(Y_test, knn_probs)
svm_auc = roc_auc_score(Y_test, y_pred)

# print('Random (chance) Prediction: AUROC = %.3f' % (r_auc))
# print('Random Forest: AUROC = %.3f' % (rf_auc))
# print('Naive Bayes: AUROC = %.3f' % (nb_auc))
# print('K Nearest Neighbors: AUROC = %.3f' % (knn_auc))
# print('SVM: AUROC = %.3f' % (svm_auc))

# # Model Accuracy: how often is the classifier correct?
# print('SVM: AUROC = %.3f' % metrics.accuracy_score(Y_test, y_pred))
#svm_fpr, svm_tpr, _ = roc_auc_score(Y_test, y_pred)
r_fpr, r_tpr, _ = roc_curve(Y_test, r_probs)
rf_fpr, rf_tpr, _ = roc_curve(Y_test, rf_probs)
nb_fpr, nb_tpr, _ = roc_curve(Y_test, nb_probs)
knn_fpr, knn_tpr, _ = roc_curve(Y_test, knn_probs)
fpr, tpr, thresholds = metrics.roc_curve(Y_test, y_pred)
plt.plot(fpr, tpr)

plt.plot(false_positive_rate,true_positive_rate,marker='.', color='green',label='NB') 
plt.plot(nb_fpr, nb_tpr, marker='.', color='red', label='RF')
plt.plot(knn_fpr, knn_tpr, marker='.', color='blue', label='DNN')
plt.plot(fpr, tpr, marker='.', color='pink', label='SVM')
plt.plot(rf_fpr, rf_tpr, marker='.', color='yellow', label='SVM')
plt.plot([0, 1], [0, 1],linestyle='--')

# Title
#plt.title('ROC Plot')
# Axis labels
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
# Show legend
plt.legend() # 
# Show plot
plt.show()